# Roofit and Roostats Bayesian calculator of upper limits

In [1]:
import ROOT
%jsroot on

Welcome to JupyROOT 6.22/03


In [2]:
infile = ROOT.TFile.Open("massupi.root")
tree = infile.Get("T")

mass = ROOT.RooRealVar("mass","m_{#Upsilon}(GeV)",8.5,9.86)
mean = ROOT.RooRealVar("mean","m_{#Upsilon}(GeV)",9.4,9.2,9.6) 
sigma = ROOT.RooRealVar("sigma","#sigma_{#Upsilon}(GeV)",0.1,0.01,0.5)


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [3]:
sig = ROOT.RooGaussian("sig","signal component",mass,mean,sigma)

In [4]:
a0 = ROOT.RooRealVar("a0","a0",0.1,-1,1)
a1 = ROOT.RooRealVar("a1","a1",0.7,-1,1) 

In [5]:
bkg = ROOT.RooChebychev("bkg","Background",mass,ROOT.RooArgList(a0,a1))
sigfrac = ROOT.RooRealVar("sigfrac","fraction of signal component",0.5,0.,1.)

In [6]:
b = ROOT.RooRealVar("b", "N_{bg}",0,19000)
s = ROOT.RooRealVar("s", "N_{sig}",0,15000)

In [7]:
model = ROOT.RooAddPdf("model", "model",ROOT.RooArgList(sig,bkg),ROOT.RooArgList(s,b))
ds = ROOT.RooDataSet("ds","ds",tree, ROOT.RooArgSet(mass))

In [8]:
fitresult = model.fitTo(ds,ROOT.RooFit.Save(),ROOT.RooFit.PrintLevel(-1));


[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (sig,bkg)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization


In [9]:
frame = mass.frame()
ds.plotOn(frame)
model.plotOn(frame)
c = ROOT.TCanvas()

n_param = fitresult.floatParsFinal().getSize();

reduced_chi_square = frame.chiSquare(n_param);
model.plotOn(frame)

#Tem que entender como adapta isso aqui pra colocar as infos direitinho na tela.
'''paveText = ROOT.TPaveText(0.5,0.5,0.9,0.9,"brNDC");
paveText.SetFillColor(ROOT.kWhite);
paveText.SetFillStyle(1001);
paveText.SetTextSize(0.025);
paveText.AddText("#chi^{2}/ndof = %f " ,reduced_chi_square);
paveText.AddText(ROOT.Form("Mean_{m_{#Upsilon}} = %.5f #pm %.5f GeV", mean.getVal(), mean.getError()));
paveText.AddText(ROOT.Form("#sigma_{m_{#Upsilon}} = %.5f #pm %.5f GeV", sigma.getVal(), sigma.getError()));
paveText.AddText(ROOT.Form("a0 = %.5f #pm %.5f", a0.getVal(), a0.getError()));
paveText.AddText(ROOT.Form("a1 = %f #pm %f", a1.getVal(), a1.getError()));
paveText.AddText(ROOT.Form("s = %.5f #pm %.5f", s.getVal(), s.getError()));
paveText.AddText(ROOT.Form("b = %f #pm %f", b.getVal(), b.getError()));
paveText.Draw();'''




frame.Draw()
c.Draw()

In [10]:
ws = ROOT.RooWorkspace()
getattr(ws,'import')(ds)
getattr(ws,'import')(model)
ws.Print()

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing dataset ds
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::mass
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooGaussian::sig
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sigma
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::s
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooChebychev::bkg
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::a0
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::a1
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::b

RooWorkspace()  contents

variables
---------
(a0,a1,b,mass,mean,s,sigma)

p.d.f.s
-------
RooChebychev::bkg[ x=mass coefList=(a0,a1) ] = 1.08054
RooAddPdf

In [11]:
sbModel = ROOT.RooStats.ModelConfig("sbModelConfig");
sbModel.SetWorkspace(ws);
sbModel.SetPdf(ws.pdf("model"));
sbModel.SetName("S+B Model");
sbModel.SetObservables(ws.var("mass"));
sbModel.SetParametersOfInterest(ws.var("s"));

In [12]:
nuisPar = sbModel.GetNuisanceParameters()

In [13]:
bayesianCalc = ROOT.RooStats.BayesianCalculator(ds,sbModel)
bayesianCalc.SetConfidenceLevel(0.95)
bayesianCalc.SetLeftSideTailFraction(0.5)
bayesianCalc.SetNumIters(1000)
bayesianCalc.SetIntegrationType("") 
bayesianCalc.SetScanOfPosterior(50)

In [14]:
firstPOI = sbModel.GetParametersOfInterest().first()

In [15]:
interval = bayesianCalc.GetInterval()

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_model_FOR_OBS_mass with 0 entries
[#1] INFO:Eval -- BayesianCalculator::GetPosteriorFunction :  nll value -278293 poi value = 12613.7
[#1] INFO:Eval -- BayesianCalculator::GetPosteriorFunction : minimum of NLL vs POI for POI =  12613.3 min NLL = -278293
[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- createNLL picked up cached consraints from workspace with 0 entries
[#1] INFO:Eval -- BayesianCalculator - scan posterior function in nbins = 50
[#1] INFO:Eval -- BayesianCalculator::GetInterval - found a valid interval : [12289.5 , 12975.3 ]


In [16]:
lowerLimit = interval.LowerLimit()
upperLimit = interval.UpperLimit()

print ("95% interval on {0} is : [{1},{2}]".format(firstPOI.GetName(),lowerLimit,upperLimit))

95% interval on s is : [12289.452975439533,12975.286601146343]


In [17]:
c2 = ROOT.TCanvas()
plot = bayesianCalc.GetPosteriorPlot()
plot.Draw()
c2.Draw()

In [18]:
#CALCULAR USANDO O MÉTODO FREQUENTISTA

In [19]:
ws.Print()


RooWorkspace()  contents

variables
---------
(a0,a1,b,mass,mean,s,sigma)

p.d.f.s
-------
RooChebychev::bkg[ x=mass coefList=(a0,a1) ] = 1.08054
RooAddPdf::model[ s * sig + b * bkg ] = 1.04314
RooGaussian::sig[ x=mass mean=mean sigma=sigma ] = 0.999644

datasets
--------
RooDataSet::ds(mass)

named sets
----------
S+B Model_Observables:(mass)
S+B Model_POI:(s)



In [20]:
ws.var("mean").setConstant(1)
ws.var("sigma").setConstant(1)
ws.var("a0").setConstant(1)
ws.var("a1").setConstant(1)
ws.var("s").setConstant(1)
ws.var("b").setConstant(1)
sbModel.GetName()

'S+B Model'

In [21]:
bModel = sbModel.Clone()
bModel.SetName(sbModel.GetName()+"_with_poi_0")

poi = bModel.GetParametersOfInterest().first()
bModel.SetSnapshot(poi)
poi.setVal(0)
bModel.SetSnapshot(ROOT.RooArgSet(poi))

[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot() replacing previous snapshot with name S+B Model_with_poi_0__snapshot


In [22]:
fc = ROOT.RooStats.FrequentistCalculator(ds, bModel, sbModel)
fc.SetToys(10000,500)
ac = ROOT.RooStats.AsymptoticCalculator(ds, bModel, sbModel)
ac.SetOneSided(True)
ac.SetPrintLevel(-1)

calc = ROOT.RooStats.HypoTestInverter(ac)
calc.SetConfidenceLevel(0.95)
calc.UseCLs(True)
calc.SetVerbose(False)
calc.SetFixedScan(10,0,20000)

[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] ERROR:InputArguments -- AsymptoticCalculator::Initialize - Null model needs a snapshot. Set using modelconfig->SetSnapshot(poi).
[#1] INFO:InputArguments -- HypoTestInverter ---- Input models: 
		 using as S+B (null) model     : S+B Model
		 using as B (alternate) model  : S+B Model_with_poi_0



In [23]:
calc.UseCLs(True)
calc.SetVerbose(False)

toymcs = calc.GetHypoTestCalculator().GetTestStatSampler()

profll = ROOT.RooStats.ProfileLikelihoodTestStat(sbModel.GetPdf())
profll.SetOneSided(True)
toymcs.SetTestStatistic(profll)

if not sbModel.GetPdf().canBeExtended():
    toymcs.SetNEventsPerToy(5000)
    print ('can not be extended')

npoints = 400
poimin = poi.getMin()
poimax = poi.getMax()

print("Poi min and max for scan: [", poimin, "," , poimax,"]")

calc.SetFixedScan(npoints,poimin,poimax)
r = calc.GetInterval()
upperLimit = r.UpperLimit()

print ("95% upper limit on {0} is : {1}".format(firstPOI.GetName(),upperLimit))

Poi min and max for scan: [ 0.0 , 15000.0 ]
95% upper limit on s is : 12837.172108946983
[#1] INFO:Eval -- HypoTestInverter::GetInterval - run a fixed scan
[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set
[#1] INFO:InputArguments -- Minimum of POI is 0 corresponds to alt  snapshot   - using qtilde asymptotic formulae  
[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot() replacing previous snapshot with name S+B Model__snapshot
[#0] WARNING:Minization -- AsymptoticCalculator:  Found a negative value of the qmu - retry to do the unconditional fit 
[#0] WARNING:Minization -- AsymptoticCalculator:  Found a better unconditional minimum  old NLL = -268179 old muHat 0
[#0] WARNING:Minization -- AsymptoticCalculator:  New minimum  found for                           NLL = -268288    muHat  37.594
[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot() replacing previous snapshot with name S+B Model__snapshot
[#0] WARNING:Minization -- AsymptoticCalculator:  Found a nega

In [24]:
plot = ROOT.RooStats.HypoTestInverterPlot("HTI_Result_Plot","HypoTest Scan Result",r)
c = ROOT.TCanvas("HypoTestInverter Scan")
c.SetLogy(False)
plot.Draw("CLb 2CL")
c.Draw()